## Full multi-modal chain

In [8]:
from src.langchain.chains.movie_rag import MovieRAGChain
from src. langchain.loaders import MoviePosterDocumentLoader
from src.retrievers.visual_retriever import VisualRetriever

### Building visual retriever

In [10]:
print("\n" + "="*60)
print("Visual Retriever")
print("="*60)

poster_loader = MoviePosterDocumentLoader('/Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/movie_posters.csv', max_movies=1000)   # small number since model is local
visual_docs = poster_loader.load()

visual_retriever = VisualRetriever(model_name="ViT-B/32", use_text_fusion=True, alpha=0.8)

visual_retriever.add_documents(visual_docs)


Visual Retriever
Loaded 6431 posters
Created 6079 poster docs.
Loading CLIP model: ViT-B/32
✓ CLIP loaded on cpu
✓ VisualRetriever (text_fusion=True, method=weight_average)
  Alpha: 0.8 (image=0.8, text=0.2)
Encoding 1000 posters with CLIP...
Encoding 1000 text descriptions with CLIP...
✓ Fused embeddings using weight_average method (dim=512)
✓ Added 1000 posters to index


### Building textual chain

#### RAG + HyDE + Reranking

In [11]:
# Configuration
PLOTS_PATH = "/Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/movie_plots.csv"
REVIEWS_PATH = "/Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/reviews_w_movies_full.csv"
MAX_MOVIES = 500  # Limit for faster demos

In [13]:
print("="*60)
print("RAG (HyDE + Reranking)")
print("="*60)

from src.langchain.prompts import ZERO_SHOT_QA_PROMPT


# Build chain with all features
full_chain = MovieRAGChain(
    plots_path=PLOTS_PATH,
    reviews_path=REVIEWS_PATH,
    max_movies=MAX_MOVIES,
    use_custom_retriever=True,
    use_custom_chunk=True,
    custom_prompt=ZERO_SHOT_QA_PROMPT,
    k=5,
    use_hyde=True,
    hyde_model="gpt-4o-mini",
    use_reranking=True,
    reranker_cfg={'type':'llm'},
    initial_k=20, 
)

full_chain.build()

RAG (HyDE + Reranking)
✓ MovieRAGChain initialized
  Retriever type: custom + reranking + HyDE
  LLM: gpt-4o-mini

Building RAG Pipeline

1. Loading documents...
Limiting to 500 movies
Loading plots from /Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/movie_plots.csv...
Created 383 plot docs.
  ✓ 383 plot documents
Loading reviews from /Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/reviews_w_movies_full.csv...
Created 500 review docs.
  ✓ 500 review documents
✓ Total: 883 reviews and plots documents

2. Chunking with custom func...

Chunking documents...
Chunked 883 docs → 10114 chunks using 'sentence' strategy.

Building base retriever...

3. Building custom retriever...
Loading embedding model: text-embedding-3-small (provider: openai)
✓ Model loaded (dimension: 1536)
✓ FaissDenseRetriever initialized (index_type=flat)
Generating embeddings for 10114 documents...
Embeddings generated
Saving index...
✓ Added 10114 documents to FAISS index
 

### Building end to end multi-modal retriever chain

In [ ]:
from src.langchain.chains.multimodal import MultiModalMovieRouter


In [36]:
router = MultiModalMovieRouter(full_chain, visual_retriever, classifier="llm")

# Queries
result1 = router.query("Recommend me a romantic movie with thriller vibes")

result2 = router.query("bright haappy movies with colorful posters")

result3 = router.query("Sci-fi movies with vibrant neon visuals")

Modality: text
I recommend "Closer." While it is primarily a drama, it features themes of love, betrayal, and complex relationships, which can create a thrilling atmosphere. The sharp dialogue and intense performances, especially by Natalie Portman, add to its engaging nature. However, if you're looking for something more traditional in romance with thriller elements, the information provided does not specify a perfect match.
Modality: both
Based on your request for bright, happy movies with colorful posters, here are some recommendations that align both textually and visually:

1. **Moana (2016)**: Although not listed in the provided posters, "Moana" is a quintessential example of a bright and happy movie. It features vibrant animation and a joyful storyline centered around empowerment and adventure. The colorful visuals reflect its Polynesian roots, making it visually appealing.

2. **The Emoji Movie (2017)**: This animated comedy is filled with bright colors and a cheerful atmospher

In [37]:
[doc['metadata']['movie_title'] for doc in result1['sources']]

['Possession', 'Closer', 'Foxcatcher', 'Foxcatcher', 'Foxcatcher']

In [38]:
print([doc['metadata']['movie_title'] for doc in result2['text_sources']])
print([doc['title'] for doc in result2['visual_sources']])

['Moana', 'Moana', 'Moana', 'Moana', 'Moana']
['Bull', 'The Emoji Movie', 'Kung Fu Panda 3', "I'm So Excited!", 'Pitch Perfect', 'The Oogieloves in The Big Balloon Adventure', 'Fast & Furious 6', 'The Professor', 'Ride Along 2', 'First Man']


In [39]:
print([doc['metadata']['movie_title'] for doc in result3['text_sources']])
print([doc['title'] for doc in result3['visual_sources']])

['Akira', 'Transcendence', 'Transcendence', 'Transcendence', 'Alien']
['First Man', "I'm So Excited!", 'Demon', 'Zero Days', 'Species', 'Blade Runner 2049', 'It Came from Outer Space', "Daniel Isn't Real", 'The Neon Demon', 'Bull']
